In [2]:
import random

In [3]:
class Task:
    def __init__(self, name, exec_time):
        self.name = name
        self.exec_time = exec_time

free_slots = [('5:00', '7:00'), ('18:00', '20:00')]

def generate_time_blocks(free_slots):
    time_blocks = []
    for start, end in free_slots:
        st_hr = int(start.split(':')[0])
        en_hr = int(end.split(':')[0])
        for hr in range(st_hr, en_hr):
            block_start = f"{hr}:00"
            block_end = f"{hr+1}:00"
            time_blocks.append((block_start, block_end))
    return time_blocks


In [4]:
task_list = [
    Task('Task1', 2),
    Task('Task2', 1),
    Task('Task3', 2),
    Task('Task4', 1),
    Task('Task5', 1)
]

time_blocks = generate_time_blocks(free_slots)

In [5]:
def generate_random_schedule(task_list, time_blocks):
    schedule = []
    available_blocks = time_blocks.copy()
    random.shuffle(available_blocks)

    for task in task_list:
        allocated = []
        needed = task.exec_time

        for block in available_blocks:
            if needed > 0:
                allocated.append((block, task.name))
                needed -= 1
        schedule.extend(allocated)

    return schedule


def fitness(schedule, task_list):
    task_time = sum([task.exec_time for task in task_list])
    used_time = len(schedule)
    idle_time = len(time_blocks) - used_time
    return 1 / (1 + idle_time)  # Minimize idle time

def crossover(sch1, sch2):
    point = len(sch1) // 2
    child = sch1[:point] + sch2[point:]
    return child


def mutate(schedule):
    if len(schedule) < 2:
        return schedule
    idx1 = random.randint(0, len(schedule) - 1)
    idx2 = random.randint(0, len(schedule) - 1)
    schedule[idx1], schedule[idx2] = schedule[idx2], schedule[idx1]
    return schedule

# Main Genetic Algorithm Loop
def genetic_algorithm(task_list, time_blocks, generations=100):
    population = [generate_random_schedule(task_list, time_blocks) for _ in range(10)]

    for _ in range(generations):
        population = sorted(population, key=lambda x: fitness(x, task_list), reverse=True)
        next_gen = population[:2]

        while len(next_gen) < len(population):
            parent1 = random.choice(population[:5])
            parent2 = random.choice(population[:5])
            child = crossover(parent1, parent2)
            child = mutate(child)
            next_gen.append(child)

        population = next_gen

    best_schedule = max(population, key=lambda x: fitness(x, task_list))
    return best_schedule

In [6]:
final_schedule = genetic_algorithm(task_list, time_blocks)

print("\nOptimized Task Schedule:")
for block, task in final_schedule:
    print(f"{block[0]} - {block[1]} --> {task}")



Optimized Task Schedule:
5:00 - 6:00 --> Task1
18:00 - 19:00 --> Task1
5:00 - 6:00 --> Task2
5:00 - 6:00 --> Task3
18:00 - 19:00 --> Task3
5:00 - 6:00 --> Task4
5:00 - 6:00 --> Task5
